In [14]:
import os
import subprocess
import sys

try:
    from bioservices import BioModels
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "bioservices"])
    from bioservices import BioModels

import json
import zipfile as z
s = BioModels()


WARNING [bioservices.BioModels:130]:  The URL (https://www.ebi.ac.uk/biomodels) provided cannot be reached.
INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


In [11]:
def download_biomodels(directory: str, ids_agreg: list, num_per_download=100):
    if num_per_download > 100:
        raise ValueError("Maximum number of models that can be downloaded at a time is 100")

    if not os.path.exists(directory):
        os.makedirs(directory)

    filenames = []
    total_models = len(ids_agreg)
    start = 0

    while start < total_models:
        end = min(start + num_per_download, total_models)
        fname = os.path.join(directory, f"Biomodels_{start + 1}_{end}.zip")
        filenames.append(fname)

        if os.path.isfile(fname):
            os.remove(fname)

        s.search_download(ids_agreg[start:end], output_filename=fname)
        print(f"Downloaded models from ID {start + 1} to {end} into {fname}")

        start = end

    with z.ZipFile(filenames[0], 'a') as z1:
        for fname in filenames[1:]:
            with z.ZipFile(fname, 'r') as zf:
                for n in zf.namelist():
                    z1.writestr(n, zf.open(n).read())

    biomodels_zip = os.path.join(directory, "biomodels.zip")
    if not os.path.isfile(biomodels_zip):
        os.rename(filenames[0], biomodels_zip)

    for fname in filenames[1:]:
        try:
            os.remove(fname)
        except Exception:
            continue

    return biomodels_zip


In [16]:
"""
NEEDS TO BE FIXED !!
"""
directory = "/Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML"
ids_file = "/Users/guillaume.souede/PycharmProjects/immunogit/src/ids_agreg.json"
with open(ids_file, "r") as f:
    ids_agreg = json.load(f)
download_biomodels(directory, ids_agreg)

ACTIVATE_DOWNLOAD = True
if ACTIVATE_DOWNLOAD:
    download_biomodels(directory, ids_agreg)

INFO    [bioservices.BioModels:240]:  /Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML/Biomodels_1_68.zip


Downloaded models from ID 1 to 68 into /Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML/Biomodels_1_68.zip


INFO    [bioservices.BioModels:240]:  /Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML/Biomodels_1_68.zip


Downloaded models from ID 1 to 68 into /Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML/Biomodels_1_68.zip
